In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np
import sqlite3

**Research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).**

In [2]:
#Checking for Vanderbilt
query = """
SELECT DISTINCT "NPI"
FROM npi_nppes
WHERE "Provider Organization Name (Legal Business Name)" LIKE '%Vanderbilt%'
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    finding_vanderbilt = pd.read_sql(query, db) 

In [3]:
#creating a tuple to be used in the query to get all the npi's belonging to Vanderbilt organizations.
vanderbilt_npi = tuple(finding_vanderbilt['NPI'].tolist())

In [14]:
#Locating all providers that have sent a small amount of referrals to Vanderbilt.
query = f"""
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "CBSA" = '34980' AND to_npi in {vanderbilt_npi}
GROUP BY 1,2,3
ORDER BY 5;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    to_vanderbilt = pd.read_sql(query, db)

In [15]:
to_vanderbilt = to_vanderbilt.rename(columns = {"NPI" : "referring_npi","Provider Organization Name (Legal Business Name)" : "referring_provider_organization", "Fullname" : "referring_provider_name", "Amount of Patients" : "referred_to_vanderbilt", "Total_referrals" : "referrels_to_Vanderbilt"})

In [16]:
#Finding TriStar Centennial Medical Center total organizations
query = """
SELECT "Provider Organization Name (Legal Business Name)","NPI", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Provider Organization Name (Legal Business Name)" LIKE '%TriStar%'
GROUP BY 1
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    finding_tristar_all = pd.read_sql(query, db)

In [7]:
tristar_npi = tuple(finding_tristar_all['NPI'].tolist())

In [17]:
query = f"""
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "CBSA" = '34980' AND to_npi in {tristar_npi}
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    to_tristar = pd.read_sql(query, db)
    
to_tristar

,NPI,Provider Organization Name (Legal Business Name),Fullname,Amount of Patients,Total_referrals
0,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,3529,9300
1,1861478489,RADIOLOGY ALLIANCE PC,None,3132,5248
2,1245393057,CENTENNIAL HEART LLC,None,2541,4052
3,1003863580,"ASSOCIATED PATHOLOGISTS, LLC",None,1884,3167
4,1184620460,None,JAMES ANDERSON MD,805,2882
...,...,...,...,...,...
202,1275577728,None,DANIEL SCOKIN M.D.,52,52
203,1851478747,None,BEATA PANZEGRAU MD,47,52
204,1083865976,None,"JAMES MASSEY M.D., PH.D.",51,51
205,1275536468,None,MUHAMMAD ASAD MD,45,51


In [18]:
to_tristar = to_tristar.rename(columns = {"NPI" : "referring_npi","Provider Organization Name (Legal Business Name)" : "referring_provider_organization", "Fullname" : "referring_provider_name", "Amount of Patients" : "referred_to_tristar", "Total_referrals" : "referrels_to_tristar"})

In [19]:
to_tristar

,referring_npi,referring_provider_organization,referring_provider_name,referred_to_tristar,referrels_to_tristar
0,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,3529,9300
1,1861478489,RADIOLOGY ALLIANCE PC,None,3132,5248
2,1245393057,CENTENNIAL HEART LLC,None,2541,4052
3,1003863580,"ASSOCIATED PATHOLOGISTS, LLC",None,1884,3167
4,1184620460,None,JAMES ANDERSON MD,805,2882
...,...,...,...,...,...
202,1275577728,None,DANIEL SCOKIN M.D.,52,52
203,1851478747,None,BEATA PANZEGRAU MD,47,52
204,1083865976,None,"JAMES MASSEY M.D., PH.D.",51,51
205,1275536468,None,MUHAMMAD ASAD MD,45,51


In [20]:
merged_df = pd.merge(to_tristar, to_vanderbilt, on ='referring_npi', how ='inner')

In [22]:
merged_df = merged_df.drop(['referring_provider_organization_y', 'referring_provider_name_y', 'referred_to_tristar', 'referred_to_vanderbilt'], axis=1)
merged_df = merged_df.rename(columns={
    'referring_provider_organization_x': 'referring_provider_organization',
    'referring_provider_name_x': 'referring_provider_name'
})


In [23]:
merged_df = merged_df.replace('None', pd.NA)
merged_df['referring_provider_or_organization']= merged_df['referring_provider_name'].fillna(merged_df['referring_provider_organization'])
merged_df = merged_df.drop(['referring_provider_organization', 'referring_provider_name'], axis = 1)

In [24]:
merged_df

,referring_npi,referrels_to_tristar,referrels_to_Vanderbilt,referring_provider_or_organization
0,1861478489,5248,20983,RADIOLOGY ALLIANCE PC
1,1245393057,4052,5765,CENTENNIAL HEART LLC
2,1003863580,3167,1362,"ASSOCIATED PATHOLOGISTS, LLC"
3,1184620460,2882,312,JAMES ANDERSON MD
4,1215932413,2814,1417,"ANESTHESIA MEDICAL GROUP, PC"
...,...,...,...,...
73,1497828321,55,5804,TENNESSEE RETINA PC
74,1922331842,55,557,"PHYSICIAN SERVICES OF MIDDLE TENNESSEE, LLC"
75,1447571658,54,2978,SUMNER REGIONAL MEDICAL CENTER LLC
76,1487654927,54,793,COUNTY OF SUMNER


***Consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?***

In [27]:
merged_npi = tuple(merged_df['referring_npi'].tolist())

In [48]:
# family medicine doctors that vanderbilt should reach out to get their patient referrals.
query = f"""
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Family Medicine' AND "CBSA" = '34980' AND to_npi in {tristar_npi} AND from_npi in {merged_npi}
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    fam_med_to_vanderbilt = pd.read_sql(query, db)

In [49]:
fam_med_to_vanderbilt = fam_med_to_vanderbilt.replace('None', pd.NA)
fam_med_to_vanderbilt['provider_name']= fam_med_to_vanderbilt['Fullname'].fillna(fam_med_to_vanderbilt['Provider Organization Name (Legal Business Name)'])
fam_med_to_vanderbilt = fam_med_to_vanderbilt.drop(['Provider Organization Name (Legal Business Name)', 'Fullname'], axis = 1)
fam_med_to_vanderbilt

,NPI,Amount of Patients,Total_referrals,provider_name
0,1184620460,805,2882,JAMES ANDERSON MD
1,1568464873,156,216,DICKSON MEDICAL ASSOCIATES PC


In [46]:
# Orthopaedic surgeons that vanderbilt should reach out to get their patient referrals.
query = f"""
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Orthopaedic Surgery' AND "CBSA" = '34980' AND to_npi in {tristar_npi} AND from_npi in {merged_npi}
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    ortho_surg_to_vanderbilt = pd.read_sql(query, db)
    

In [47]:
ortho_surg_to_vanderbilt = ortho_surg_to_vanderbilt.replace('None', pd.NA)
ortho_surg_to_vanderbilt['provider_name']= ortho_surg_to_vanderbilt['Fullname'].fillna(ortho_surg_to_vanderbilt['Provider Organization Name (Legal Business Name)'])
ortho_surg_to_vanderbilt = ortho_surg_to_vanderbilt.drop(['Provider Organization Name (Legal Business Name)', 'Fullname'], axis = 1)
ortho_surg_to_vanderbilt

,NPI,Amount of Patients,Total_referrals,provider_name
0,1538153937,148,228,TENNESSEE ORTHOPAEDIC ALLIANCE PA
1,1972577351,47,83,PREMIER ORTHOPAEDICS & SPORTS MEDICINE PLC


In [ ]:
#finding the family medicine providers in the Nashville area that refers the most patients.
query = """
SELECT "NPI","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, "Classification", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Family Medicine' AND "CBSA" = '34980' AND "Entity Type Code" = 01
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    family_med = pd.read_sql(query, db)
    
family_med

In [ ]:
##finding the orthopaedic surgeon providers in the Nashville area that refers the most patients.
query = """
SELECT "NPI","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, "Classification", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Orthopaedic Surgery' AND "CBSA" = 34980 AND "Entity Type Code" = 01
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    ortho_surgeon = pd.read_sql(query, db)
    
ortho_surgeon

In [ ]:
#query getting the data needed to create a csv for neo4j
query = """
SELECT * 
FROM npi_nppes as np
join hop_npi as n
on np.NPI = n.from_npi
WHERE np.CBSA = 34980

"""
with sqlite3.connect('../data/npi.sqlite') as db:
    nppes_nashville = pd.read_sql(query, db)

In [ ]:
#Creating a csv to load into neo4j
nppes_nashville.to_csv("../data/nppes_nashville.csv")

In [ ]:
nppes_nashville 

In [ ]:
# query = """
# SELECT COUNT(*)
# FROM npi_nppes 

# """
# with sqlite3.connect('../data/npi.sqlite') as db:
#     check = pd.read_sql(query, db)
#check

In [ ]:
#table with the correct Vanderbilt organization
#query = """
#SELECT *
#FROM npi_nppes
#WHERE "Provider Organization Name (Legal Business Name)" = 'VANDERBILT UNIVERSITY MEDICAL CENTER'
#"""
#with sqlite3.connect('../data/npi.sqlite') as db:
    #The_vanderbilt = pd.read_sql(query, db)
    
#The_vanderbilt

In [ ]:
#Finding TriStar Centennial Medical Center ONLY in Nashville area
# query = """
# SELECT "Provider Organization Name (Legal Business Name)","NPI","Provider First Line Business Practice Location Address", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
# FROM npi_nppes AS nppes
# INNER JOIN hop_npi AS npi
# ON nppes.NPI = npi.from_npi
# WHERE "CBSA" = '34980' AND "Provider Organization Name (Legal Business Name)" LIKE '%TriStar%'
# GROUP BY 1,2,3
# """
# with sqlite3.connect('../data/npi.sqlite') as db:
#     finding_tristar = pd.read_sql(query, db)
    
# finding_tristar